### [Facial Keypoints Detection](https://www.kaggle.com/c/facial-keypoints-detection)

**The objective of this task is to predict keypoint positions on face images.**

So our input is image of person and goals is here to predict the 30 keypoints on the images

In [ ]:
!pip install visualkeras

# Import all essential libraries and read the data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import visualkeras


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn import model_selection
from sklearn import metrics
df = pd.read_csv('../input/facial-keypoints-detection/training.zip', compression='zip', header=0, sep=',', quotechar='"')


In [ ]:
all_features=df.columns
print('---------No. of columns------------\n')
print(list(all_features))
print('-------------DataFrame-------------\n')
print(df.iloc[0])

Percent of missing data

In [ ]:
# determine the threshold for missing values
def percent_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().mean()*100,2)})
    
    return dict_x

missing = percent_missing(df)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:10]

In [ ]:
print('-----------DataFrame shape--------')
print(df.shape[0])
print('-----------DataFrame shape after dropping all NA values--------')
print(df.dropna().shape[0])

Clearly we cannot drop the row with NA vlaues as we will loos the 3000 rows.

We impute the data with forward fill method

In [ ]:
df.fillna(method = 'ffill',inplace = True)

In [ ]:
#Helper function
def preprocessing_Images(data):
    data=data.apply(lambda x:np.fromstring(x,dtype=int,sep=' ').reshape(96,96))
    
    #Normalize the image
    data=data/255
    
    # empty array to feed the model of shape(96,96,1)
    temp= np.empty((len(data),96,96,1))

    #expanding dimensions to (96,96,1)
    for i in range(len(data)):
        temp[i,]=np.expand_dims(data[i],axis=2)
    return temp

def images_vis(x,y,loc,y_pred,point_show=True):
    plt.imshow(x[loc], cmap = 'gray')
    if point_show==True:
        for j in range(0, 28, 2):
            plt.plot(y.iloc[loc][j], y.iloc[loc][j+1], 'bo',label='Actual values')
            plt.plot(y_pred[loc][j], y_pred[loc][j+1], 'rx',label='Predicted values')
            
def images_vis_train(x,y,loc,point_show=True):
    plt.imshow(x[loc], cmap = 'gray')
    if point_show==True:
        for j in range(0, 28, 2):
            plt.plot(y.iloc[loc][j], y.iloc[loc][j+1], 'bo',label='Actual values')

In [ ]:
X=preprocessing_Images(df.Image)

targets=list(df.columns)
targets.remove("Image")

y= df[targets]
# Split the data into 95 : 05 ratio
x_train,x_test, y_train, y_test=model_selection.train_test_split(X,y,test_size=0.05,random_state=42)

Sample Images

In [ ]:
n_rows=2
n_cols=5
fig = plt.figure(figsize=(24,6.6), dpi=100)
for i, idx in enumerate(np.random.randint(0, len(y_train), n_rows*n_cols)):
    axis = fig.add_subplot(n_rows, n_cols, i+1, xticks=[], yticks=[])
    images_vis_train(x=x_train,y=y_train,loc=idx,point_show=True)
    axis.set_title(f'Sample #{idx}')

# Model Building

In [ ]:
model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=(96,96,1)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(30)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'acc'])


visualkeras.layered_view(model)

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=15, batch_size=32,validation_split=0.2)

## Accuracy Graphs

In [ ]:
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot(title='Loss vs Epochs')
history_frame.loc[:, ['acc', 'val_acc']].plot(title='Accuary vs Epochs');
history_frame.loc[:, ['mae', 'val_mae']].plot(title='MAE vs Epochs');

# Predict the data

In [ ]:
test_preds = model.predict(x_test)
print(np.sqrt(metrics.mean_squared_error(y_test,test_preds)))

In [ ]:
i=4
plt.figure(figsize=(6,6))
plt.imshow(x_test[i], cmap = 'gray')
for j in range(0, 28, 2):
    plt.plot(test_preds[i][j], test_preds[i][j+1], 'rx',label='Predicted values')
    plt.plot(y_test.iloc[i][j], y_test.iloc[i][j+1], 'bo',label='Actual Values')
    
# Function add a legend   
plt.legend(["Predicted values", "Actual Values"], loc ="best") 
  
# function to show the plot 
plt.show() 

# Sample Predictions

In [ ]:
n_rows=2
n_cols=5
fig = plt.figure(figsize=(24,6.5), dpi=100)
for i, idx in enumerate(np.random.randint(0, len(y_test), n_rows*n_cols)):
    axis = fig.add_subplot(n_rows, n_cols, i+1, xticks=[], yticks=[])
    images_vis(x=x_test,y=y_test,loc=idx,y_pred=test_preds,point_show=True)
    axis.set_title(f'Sample #{idx}')
# Function add a legend   
plt.legend(["Predicted values", "Actual Values"], loc ="best") 
  
# function to show the plot 
plt.show() 